In [29]:
from dotenv import load_dotenv
import dspy

In [30]:
load_dotenv("../.env")

True

In [31]:
class QuestionAnswer(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField()


class ZeroShot(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predictor = dspy.Predict(QuestionAnswer)

    def forward(self, question):
        return self.predictor(question=question)

In [32]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct")
dspy.settings.configure(lm=turbo)

model = ZeroShot()
question = "What is the capital of Brasil?"
answer = model(question=question)
print(answer.answer)

Brasília
